In [ ]:
%load_ext autoreload
%autoreload 2
# default_exp plugin.authenticators.oauth

In [ ]:
# export 
# hide
import abc
from time import sleep
from pymemri.plugin.pluginbase import PluginBase
from pymemri.plugin.states import RUN_USER_ACTION_NEEDED, RUN_USER_ACTION_COMPLETED, RUN_FAILED
from pymemri.plugin.schema import PluginRun, Account
from pymemri.pod.client import PodClient

## OAuth Authenticator

For plugins that require OAuth, `OAuthAuthenticator` provides an easy interface to login to thirds party services. By implementing You can nest an inherited version class of `OAuthAuthenticator` in your `Plugin` item and call authenticator.authenticate().

`YourOauthAuthenticator` class should implement:
- get_oauth_url() -> str to setup OAuth endpoint
- get_tokens_from_code() -> Dict[str, str] to retrieve tokens from the service with returned OAuth code
- refresh_tokens() -> None to get new token pairs

In [ ]:
# export
# hide

class OAuthAuthenticator(metaclass=abc.ABCMeta):

    SLEEP_INTERVAL = 1.0

    def __init__(self, client, pluginRun):
        self.client = client
        self.pluginRun = pluginRun

    def authenticate(self):
        account = self.pluginRun.account[0] if len(self.pluginRun.account) > 0 else None
        if account.accessToken:
            if self.verify_access_token(account.accessToken):
                return

        tokens = None
        try:
            if not self.pluginRun.account[0].refreshToken:
                raise Exception("Refresh token is empty")
            tokens = self.refresh_tokens(self.pluginRun.account[0].refreshToken)
        except: # no account exists or expired refresh token
            url = self.get_oauth_url()
            self.present_url_to_user(url)
            code = self.poll_for_code()
            tokens = self.get_tokens_from_code(code)
            # if not tokens: raise an exception or set pluginRun.status=FAILED and pluginRun.message=ERROR_MESSAGE

        self.store_tokens(tokens)
        
    def poll_for_code(self):
        while True:
            sleep(self.SLEEP_INTERVAL)
            self.pluginRun = self.client.get(self.pluginRun.id)
            if self.pluginRun.status == RUN_USER_ACTION_COMPLETED:
                return self.pluginRun.account[0].code
            if self.pluginRun.status == RUN_FAILED:
                raise Exception(f"Error in plugin.authenticators.oauth {self.pluginRun.message}") 

    def present_url_to_user(self, url):
        # request user to visit url
        self.pluginRun.authUrl = url
        self.pluginRun.status = RUN_USER_ACTION_NEEDED
        self.pluginRun.update(self.client)               

    def store_tokens(self, tokens):
        account = self.pluginRun.account[0]
        account.accessToken = tokens['access_token']
        account.refreshToken = tokens['refresh_token'] if 'refresh_token' in tokens else None
        account.update(self.client)
        
    def get_account(self):
        return self.pluginRun.account[0]

    @abc.abstractmethod
    def get_oauth_url(self):
        raise NotImplemented()

    @abc.abstractmethod
    def get_tokens_from_code(self, code):
        """ Gets access and refresh tokens from 3rd party service
            and returns them in form:
                {
                    'access_token': '...',
                    'refresh_token': '...'
                }
        """
        raise NotImplemented()

    @abc.abstractmethod
    def refresh_tokens(self):
        """ Gets new tokens by using an existing refresh token
            and returns them in form:
                {
                    'access_token': '...',
                    'refresh_token': '...'
                }

        """
        # use self.pluginRun.account[0].refreshToken
        raise NotImplemented()

    @abc.abstractmethod
    def verify_access_token(self, token):
        """
        Check if existing token is working. If this returns True, then user interaction will not be needed.
        """
        raise NotImplemented()

## Example

Lets build an example oauth authenticator, to show its class structure

In [ ]:
class ExampleOAuthAuthenticator(OAuthAuthenticator):

    def get_oauth_url(self):
        return "https://example.com/oauth"

    def get_tokens_from_code(self, code):
        return {
            'access_token': 'dummy_access_token',
            'refresh_token': 'dummy_refresh_token'
        }

    def refresh_tokens(self, refreshToken):
        return {
            'access_token': 'refreshed_dummy_access_token',
            'refresh_token': 'refreshed_dummy_refresh_token'
        }

    def verify_access_token(self, token):
        if token:
            return True
    
    def present_url_to_user(self, *args):
        # NORMALLY YOU WOULD NOT IMPLEMENT THIS
        # mocking user interaction
        self.pluginRun.status = RUN_USER_ACTION_COMPLETED
        self.pluginRun.account[0].code = "dummy code"
        self.pluginRun.account[0].update(self.client)
        self.pluginRun.update(self.client)


In [ ]:
class MyOAuthPlugin(PluginBase):

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def run(self):
        print("running")
        self.login()
        print("Login completed!")
        
    def login(self):
        auth = ExampleOAuthAuthenticator(self.client, self.pluginRun)
        auth.authenticate()
        print(f"logged in with {auth.get_account().accessToken}")

    def add_to_schema(self):
        self.client.add_to_schema(MyItem("my name", 10))

In [ ]:
client = PodClient()

# Create a dummy account to use for authentication within the plugin
account = Account(service="my_plugin_service", identifier="username", secret="password")
# Create a run to enable plugin runtime
run = PluginRun("pymemri", "pymemri.plugin.pluginbase", "MyOAuthPlugin")
run.add_edge('account', account)
account.update(client)
run.update(client)

In [ ]:
plugin = MyOAuthPlugin(pluginRun=run, client=client)

In [ ]:
plugin.run()

### Test Plugin properties and attached items -

In [ ]:
# # hide
# from pymemri.plugin.schema import Account
# from pymemri.plugin.pluginbase import PluginRun, run_plugin_from_run_id

# client = PodClient()

# # Create a dummy account to use for authentication within the plugin
# account = Account(service="my_plugin_service", identifier="username", secret="password")
# client.create(account)

# # Create a run to enable plugin runtime
# run = PluginRun("pymemri", "pymemri.plugin.pluginbase", "MyOAuthPlugin")
# run.add_edge('account', account)
# client.create(run)
# client.update(account)

# plugin = run_plugin_from_run_id(run.id, client)

# # check if authentication worked
# assert plugin.pluginRun.account[0].identifier == "username"
# assert plugin.pluginRun.account[0].accessToken == "dummy_access_token"

# # set a state
# plugin.pluginRun.status = "test state"
# plugin.pluginRun.update(client)
# plugin.pluginRun = client.get(plugin.pluginRun.id)
# assert plugin.pluginRun.status == "test state"